In [1]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
from ultralytics import YOLO
import cv2
import tensorflow as tf
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
import json
import pickle

In [2]:
FILE_ROOT = "J:\\jupyter\\plate_recognition\\car_plate_fa_numbers"
IMAGE_PATH = FILE_ROOT + "\\images"  
ANNOTATIONS_PATH = FILE_ROOT + "\\annotations"

DATA_ROOT = "datasets\\Dataset_fa_numbers\\"
DEST_IMAGES_PATH = "train\\images"
DEST_LABELS_PATH = "train\\labels"


In [3]:
def resize_with_aspect_ratio(image, new_height):
    # Calculate the aspect ratio
    aspect_ratio = image.shape[1] / image.shape[0]
    
    # Calculate the new width based on the desired height and aspect ratio
    new_width = int(new_height * aspect_ratio)
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))
    
    return resized_image
    
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1, x2, y1, y2 = box["x1"], box["x2"], box["y1"], box["y2"]
    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]
    
def save_label(img_jpg_file_name, size, img_box):
    classes = ['license']
    save_file_name = DATA_ROOT + DEST_LABELS_PATH + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:                  
        cls_num = encoder.transform([box["class_name"]])[0]
        new_box = cord_converter(size, box ) 

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()

def get_bbox_from_xml(box):
    corners=["xmin","ymin","xmax","ymax"]
    corners_numbers=[]
    for c in corners:
        data=int(float(box.getElementsByTagName(c)[0].childNodes[0].data))
        corners_numbers.append(data)
    return corners_numbers
    
        
    save_file(img_xml_file, [img_w, img_h], img_box)

In [5]:
#getting plates
def r():
    files = os.listdir(ANNOTATIONS_PATH)
    plates=[]
    for file in files:
        # print("file name: ", file)
        file_name = file.split(".")[0]
        xml_path = ANNOTATIONS_PATH + '/' + file_name +".xml"
        img_path=IMAGE_PATH +"/" + file_name +".jpg"
        dom = parse(xml_path)
        root = dom.documentElement
        img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
        img_size = root.getElementsByTagName("size")[0]
        objects = root.getElementsByTagName("object")
        img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
        img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    
        img=cv2.imread(img_path)
        for box in objects:
            cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
            x1,y1,x2,y2=get_bbox_from_xml(box)
            if cls_name=="کل ناحیه پلاک":
                plates.append({"img":img[y1:y2,x1:x2],
                               "x1":x1, "y1":y1, "x2":x2, "y2":y2,
                               "numbers":[] 
                              })
            else:
                last_plate=plates[-1]
                x1=abs( x1 - last_plate["x1"])
                y1=abs(y1 - last_plate["y1"])
                x2=abs(x2 - last_plate["x1"])
                y2=abs(y2 - last_plate["y1"])
                last_plate["numbers"].append({"class_name":cls_name, "x1":x1, "y1":y1, "x2":x2, "y2":y2})
                
            # print("box:(xmin,ymin,xmax,ymax)", x1, y1, x2, y2)
    return plates
    for plate in plates:
        plate_copy=plate["img"].copy()
        for number in  plate["numbers"]:
            print("class:",number["class_name"])
            pt1=(number["x1"],number["y1"])
            pt2=(number["x2"],number["y2"])
            print("pt1:",pt1,"pt2:",pt2)
            cv2.rectangle(plate_copy,pt1,pt2,(0,255,255))
           
        cv2.imshow("win",resize_with_aspect_ratio(plate_copy,100) )
        key=cv2.waitKey(0)
        cv2.destroyAllWindows()
        if chr(key)=="q":
            return
plates=r()

In [33]:
unique_class_names = set()

# Iterate through the array
for element in plates:
    # Get the 'numbers' list from each element
    numbers_list = element['numbers']
    
    # Iterate through the 'numbers' list
    for number in numbers_list:
        class_name = number['class_name']
        # Add the class_name to the set
        unique_class_names.add(class_name)

# Convert the set to a list if needed

# Print the unique class names
print(unique_class_names)

j=json.dumps(list(unique_class_names))
with open("numbers.json",mode="w") as file:
    file.write(j)

{'4', '0', 'ج', 'ه\u200d', 'ط', '7', 'ت', '9', 'د', 'ب', 'ث', '5', 'الف', 'و', '2', 'ن', '1', 'پ', 'ق', 'ص', 'ژ (معلولین و جانبازان)', 'س', '3', 'ل', '6', 'ظ', 'م', '8', 'ی', 'ع'}


In [75]:
with open("plates_numbers.json",mode="w") as file:
    f=json.dumps(plates)
    file.write(f)

In [52]:
encoder=LabelEncoder()
encoder.fit(list(unique_class_names))

LabelEncoder()

In [37]:
file=open("numbers.json")
unique_numbers=json.loads(file.read())
file.close()
file=open("encoder.pkl","rb")
encoder=pickle.load(file)
file.close()

In [51]:
unique_numbers=sorted(unique_numbers,key=lambda x:encoder.transform([x])[0])

In [81]:
for i,plate in enumerate(plates):
    plate["img"]=np.array(plate["img"])
    name=str(i)
    path=DATA_ROOT+ DEST_IMAGES_PATH +"/" +name+".jpg"
    h,w=plate["img"].shape[:2]
    size=w,h
    result=cv2.imwrite(path,np.array(plate["img"]) )
    save_label(name,size,plate["numbers"])
    print("\r" +str(i),end="")


datasets\Dataset_fa_numbers\train\labels/0.txt
0datasets\Dataset_fa_numbers\train\labels/1.txt
1datasets\Dataset_fa_numbers\train\labels/2.txt
2datasets\Dataset_fa_numbers\train\labels/3.txt
3datasets\Dataset_fa_numbers\train\labels/4.txt
4datasets\Dataset_fa_numbers\train\labels/5.txt
5datasets\Dataset_fa_numbers\train\labels/6.txt
6datasets\Dataset_fa_numbers\train\labels/7.txt
7datasets\Dataset_fa_numbers\train\labels/8.txt
8datasets\Dataset_fa_numbers\train\labels/9.txt
9datasets\Dataset_fa_numbers\train\labels/10.txt
10datasets\Dataset_fa_numbers\train\labels/11.txt
11datasets\Dataset_fa_numbers\train\labels/12.txt
12datasets\Dataset_fa_numbers\train\labels/13.txt
13datasets\Dataset_fa_numbers\train\labels/14.txt
14datasets\Dataset_fa_numbers\train\labels/15.txt
15datasets\Dataset_fa_numbers\train\labels/16.txt
16datasets\Dataset_fa_numbers\train\labels/17.txt
17datasets\Dataset_fa_numbers\train\labels/18.txt
18datasets\Dataset_fa_numbers\train\labels/19.txt
19datasets\Dataset_fa_

In [107]:
def test_labels():
    labels_path=DATA_ROOT+"\\"+DEST_LABELS_PATH
    for txt_file in os.listdir(labels_path):
        with open(labels_path+"\\"+txt_file)as file:
            img=cv2.imread(DATA_ROOT+"\\"+DEST_IMAGES_PATH+"\\"+txt_file.split(".")[0]+".jpg")
            numbers=file.read().splitlines()
            for number in numbers:
                label,x,y,w,h=np.float64(number.split(" "))
                x1=x-w/2
                x2=x1+w
                y1=y-h/2
                y2=y1+h
                x1=int(x1*img.shape[1])
                x2=int(x2*img.shape[1])
                y1=int(y1*img.shape[0])
                y2=int(y2*img.shape[0])
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,255))
                
                cv2.imshow("win",img)
                key=cv2.waitKey(0)
                cv2.destroyAllWindows()
                if chr(key)=="q":
                    return
test_labels()

In [4]:
load_path="J:/jupyter/plate_recognition/runs/detect/train11/weights/best.pt"#YOU SHOULD CHANGE THIS EVERYTIME
model=YOLO(load_path,task="detect")


In [85]:
results=model.train(data="numbers.yaml", epochs=10, imgsz=512)

New https://pypi.org/project/ultralytics/8.0.190 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.188  Python-3.9.17 torch-2.0.1+cpu CPU (Intel Core(TM) i7-4790K 4.00GHz)
engine\trainer: task=detect, mode=train, model=J:/jupyter/plate_recognition/runs/detect/train9/weights/best.pt, data=numbers.yaml, epochs=10, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_wi

In [5]:
model.names

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'الف',
 11: 'ب',
 12: 'ت',
 13: 'ث',
 14: 'ج',
 15: 'د',
 16: 'س',
 17: 'ص',
 18: 'ط',
 19: 'ظ',
 20: 'ع',
 21: 'ق',
 22: 'ل',
 23: 'م',
 24: 'ن',
 25: 'ه',
 26: 'و',
 27: 'پ',
 28: 'ژ',
 29: 'ی'}

In [43]:
name="147.jpg"
test_path="J:/jupyter/plate_recognition/car_plate_fa_numbers/test/"  + name
img=cv2.imread(test_path)
pad=16
img=cv2.copyMakeBorder(img,pad,pad,pad,pad, cv2.BORDER_CONSTANT)
# preds=model.val()
preds=model.predict([img],save=True)
bboxes,classes =zip(*sorted(list(zip(preds[0].boxes.xywh,preds[0].boxes.cls)),key=lambda x:x[0][0]))
classess_labled=list(map(lambda x:model.names[x.item()] ,classes))
for cls in classess_labled:
    print(cls)
cv2.imshow("win",img)
cv2.waitKey()
cv2.destroyAllWindows()


0: 224x512 1 4, 1 5, 1 7, 1 8, 3 9s, 1 , 58.8ms
Speed: 4.0ms preprocess, 58.8ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 512)
Results saved to runs\detect\predict21


4
7
م
5
9
8
9
9
